In [1]:
from os.path import join
from os import listdir
import pandas as pd
import numpy as np
import json

if False:
    # Read data
    GENRES = ["blues", "gospel", "rap", "country", "rock"]
    DATA_DIR = "/n/fs/guoweis-18iw/get_data/lyrics"

    df = pd.DataFrame(np.nan, index=[], columns=['artist', 'title', 'album', 'year', 'lyrics', 'genre'])

    ct = 0
    for genre in GENRES:
        genre_dir = join(DATA_DIR, genre)
        fns = listdir(genre_dir)
        for i, fn in enumerate(fns):
            if i % 10 == 0:
                print("Done with " + str(i) + " of " + str(len(fns)) + " files.")
            fp = join(genre_dir, fn)
            data_str = open(fp).read()
            data = json.loads(data_str)
            songs_data = data["songs"]
            for j, song in enumerate(songs_data):
                df.loc[ct, "genre"] = genre
                for key in song.keys():
                    if key == "raw" or key == "image":
                        continue
                    df.loc[ct, key] = song[key]
                ct += 1
        df.to_pickle(genre + ".data")


In [ ]:
# Read Data
ALL_DATA_FN = "all.data"
df = pd.read_pickle(ALL_DATA_FN)

In [ ]:
print((df.loc[:,"genre"])